In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd

import datetime as dt

from ipywidgets import interact
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sova.market_data import generate_market_data
from sova.asset import StockOption, OptionType
from sova.portfolio import Portfolio, Trade

import importlib
import sova
importlib.reload(sova.asset);

In [4]:
@interact(p_val=(0, .1, .0001), q_val=(0, .1, .0001), a0=(0., 1e-3, 1e-5))
def plot_asset_vol_returns(p_val, q_val, a0):
    print(a0)
    print(p_val)
    print(q_val)

    n_obs = 500
    p_arr = np.array([p_val])
    q_arr = np.array([q_val])
    md = generate_market_data(a0, p_arr, q_arr, n_obs=n_obs)
    x_arr = md.index
    fig1 = go.Figure(
        data=go.Scatter(x=x_arr, y=md.asset_price, line_color='deepskyblue'))
    fig1.update_layout(title='Asset price')
    fig1.add_shape(type="line",
                   x0=md.index[0],
                   y0=md.asset_price[0],
                   x1=md.index[-1],
                   y1=md.asset_price[0],
                   line=dict(
                       color="MediumPurple",
                       width=4,
                       dash="dot",
                   ))
    fig1.show()

    opt_call = StockOption(amount=1,
                           expiry=pd.Series(md.index).max(),
                           strike=md.asset_price[0],
                           sigma=md.asset_volatility[0],
                           option_type=OptionType.CALL)
    pf = Portfolio(trades=[Trade(x_arr[0], md.asset_price[0], opt_call)])
    pf_price = md.apply(lambda r: pf.current_price(
        stock_price=r.asset_price, market_time=r.name.to_pydatetime()),
                        axis=1)
    pf_delta = md.apply(lambda r: pf.current_delta(
        stock_price=r.asset_price, market_time=r.name.to_pydatetime()),
                        axis=1)

    fig2 = go.Figure(data=go.Scatter(x=x_arr, y=pf_price))
    fig2.update_layout(title='Portfolio PV')
    fig2.show()

    fig3 = go.Figure(data=go.Scatter(x=x_arr, y=pf_delta))
    fig3.update_layout(title='Portfolio Delta')
    fig3.show()

interactive(children=(FloatSlider(value=0.05, description='p_val', max=0.1, step=0.0001), FloatSlider(value=0.…